In [99]:
import cv2
import numpy as np
from math import ceil,floor

In [100]:
vidObj = cv2.VideoCapture('movie.mov') 

frames = []
input_frames = []

count = 0
success = 1

while success: 
    success, image = vidObj.read()     
    if image is None:
        break
    
    
    frames.append(image)
    input_frames.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)/255)

In [101]:
reference = cv2.imread('reference.jpg', cv2.IMREAD_GRAYSCALE)/255

In [102]:
def ExhaustiveSearch(t,r):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    
    for m in range(I-M+1): #
        for n in range(J-N+1): #
            current_correlation = 0
            for i in range(m,m+M-1):
                for j in range(n,n+N-1):
                    current_correlation += t[i][j]*r[i-m][j-n]
                    
            if(m%100==0):
                if(n%100==0):
                    print(m,n,current_correlation)
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
    
    return best_node,max_correlation

In [103]:
#[X,Y],max_correlation = ExhaustiveSearch(input_frames[0],reference)
[X,Y],max_correlation = [247,137], 3588.946220684356
M,N = reference.shape[0],reference.shape[1]

print(X,Y,max_correlation)

247 137 3588.946220684356


In [104]:
def ExhaustiveSearchUsingCenter(t,r,X,Y,p,previous_correlation):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    
    for m in range(X-p,X+p): #
        for n in range(Y-p,Y+p): #
            current_correlation = 0
            for i in range(m,m+M-1):
                for j in range(n,n+N-1):
                    current_correlation += t[i][j]*r[i-m][j-n]
                    
            if(previous_correlation>=current_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
                break
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
    
    return best_node,max_correlation

In [132]:
def twoDimensionalLogarithmicSearch(t,r,X,Y,p):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [X,Y]
    
    max_correlation = 10e10
    
    div = p
    
    while True:
        arr = [floor(-div/2),0,ceil(div/2)]
                
        #print(max_correlation)
        
        for m in arr:
            for n in arr:                
                x,y = X+m,Y+n
                
                if(x==X and y==Y):
                    continue
                
                current_correlation = 0
                
                for i in range(x,x+M-1):
                    for j in range(y,y+N-1):
                        current_correlation += t[i][j]*r[i-x][j-y]
                
                if(current_correlation<max_correlation):
                    max_correlation = current_correlation
                    best_node = [x,y]
                
        [X,Y] = best_node

        div = int(div/2)
        
        if(div<1):
            break
    
    return best_node,max_correlation


In [139]:
def outline_creator(frame,center_x,center_y,M,N):
    for i in range(center_x-5,center_x+M+5):
        frame[i][center_y] = [0,0,255]
        frame[i][center_y+N+5] = [0,0,255]
    
    for j in range(center_y,center_y+N+5):
        frame[center_x-5][j] = [0,0,255]
        frame[center_x+M+5][j] = [0,0,255]
            
    return frame    

In [140]:
def output_frames(frames,input_frames,reference,X,Y,p,first_frame_output,max_correlation):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation = ExhaustiveSearchUsingCenter(input_frames[i],reference,X,Y,p,max_correlation)
        print([X,Y],max_correlation)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output

In [142]:
#twoDimensionalLogarithmicSearch(input_frames[1],reference,247,137,7)

def output_frames_2DLogSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation = twoDimensionalLogarithmicSearch(input_frames[i],reference,X,Y,p)
        print([X,Y],max_correlation)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output

In [143]:
first_frame_output = outline_creator(frames[0],X,Y,M,N)
#output_frames = output_frames(frames,input_frames,reference,X,Y,2,first_frame_output,max_correlation)

output_frames = output_frames_2DLogSearch(frames,input_frames,reference,X,Y,2,first_frame_output)

[247, 138] 3587.488273740882
[247, 139] 3586.9623529411797
[247, 140] 3589.9724413687104
[247, 141] 3589.078216070753
[247, 142] 3590.1891580161478
[247, 143] 3589.657623990784
[247, 144] 3589.7780545943915
[247, 145] 3588.946220684356
[247, 146] 3587.488273740882
[247, 147] 3586.9623529411797
[247, 148] 3589.9724413687104
[247, 149] 3589.078216070753
[247, 150] 3590.1891580161478
[247, 151] 3589.657623990784
[247, 152] 3589.7780545943915
[247, 153] 3588.946220684356
[247, 154] 3587.488273740882
[247, 155] 3586.9623529411797
[247, 156] 3589.9724413687104
[247, 157] 3589.078216070753
[247, 158] 3590.1891580161478
[247, 159] 3589.657623990784
[247, 160] 3589.7780545943915
[247, 161] 3588.946220684356
[247, 162] 3587.488273740882
[247, 163] 3586.9623529411797
[247, 164] 3589.9724413687104
[247, 165] 3589.078216070753
[247, 166] 3590.1891580161478
[247, 167] 3589.657623990784
[247, 168] 3589.7780545943915
[247, 169] 3588.946220684356
[247, 170] 3587.488273740882
[247, 171] 3586.96235294117

[154, 326] 3588.4271434063826
[153, 326] 3589.58094579008
[152, 326] 3589.3857285659356
[151, 326] 3590.1891580161478
[150, 326] 3590.2424452133832
[149, 326] 3590.150372933488
[148, 326] 3589.2700346020806
[147, 326] 3587.284875048064
[146, 326] 3588.4271434063826
[145, 326] 3589.58094579008
[144, 326] 3589.3857285659356
[143, 326] 3590.1891580161478
[142, 326] 3590.2424452133832
[141, 326] 3590.150372933488
[140, 326] 3589.2700346020806
[139, 326] 3587.284875048064
[138, 326] 3588.4271434063826
[137, 326] 3589.58094579008
[136, 326] 3589.3857285659356
[135, 326] 3590.1891580161478
[134, 326] 3590.2424452133832
[133, 326] 3590.150372933488
[132, 326] 3589.2700346020806
[131, 326] 3587.284875048064
[130, 326] 3588.4271434063826
[129, 326] 3589.58094579008
[128, 326] 3589.3857285659356
[127, 326] 3590.1891580161478
[126, 326] 3590.2424452133832
[125, 326] 3590.150372933488
[124, 326] 3589.2700346020806
[123, 326] 3587.284875048064
[122, 326] 3588.4271434063826
[121, 326] 3589.5809457900

[58, 137] 3587.5606459054184
[58, 137] 3587.5606459054184
[59, 137] 3587.92389081123
[60, 137] 3589.0231141868553
[61, 137] 3590.4999000384555
[62, 137] 3589.545636293741
[63, 137] 3588.946220684356
[64, 137] 3588.174086889669
[65, 137] 3586.882460592085
[66, 137] 3587.5606459054184
[67, 137] 3587.92389081123
[68, 137] 3589.0231141868553
[69, 137] 3590.4999000384555
[70, 137] 3589.545636293741
[71, 137] 3588.946220684356
[72, 137] 3588.174086889669
[73, 137] 3586.882460592085
[74, 137] 3587.5606459054184
[75, 137] 3587.92389081123
[76, 137] 3589.0231141868553
[77, 137] 3590.4999000384555
[78, 137] 3589.545636293741
[79, 137] 3588.946220684356
[80, 137] 3588.174086889669
[81, 137] 3586.882460592085
[82, 137] 3587.5606459054184
[83, 137] 3587.92389081123
[84, 137] 3589.0231141868553
[85, 137] 3590.4999000384555
[86, 137] 3589.545636293741
[87, 137] 3588.946220684356
[88, 137] 3588.174086889669
[89, 137] 3586.882460592085
[90, 137] 3587.5606459054184
[91, 137] 3587.92389081123
[92, 137] 3

In [144]:
cap = cv2.VideoCapture('movie.mov')
fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output.avi', fourcc, 760/13, (int(cap.get(3)),int(cap.get(4))))


for frame in output_frames:
    out.write(frame)
cap.release()
out.release()